## File structure
accident
## Steps
### General
对于一个事故详情页面：
    从正文借助关键词判断是否所需事故，标记优先级。高优先级后续分析pdf可参考
    下载pdf以便后续分析

pdf分析：
提取文字和表格文字，通过关键词爬取所需属性。
由于pdf中表述不一定统一，每个属性可以提取为：包含关键词的上下文。这样的上下文可能有多个，均可列出，最后人工提取。
如果发现高度规律，再进行自动提取。
    
### Detailed
Start at "按事故类型分类"
for: each accident type, 
    simulte click to load coresponding page
    while: "active" != "last".total
        get article list
        for: each article in article list
            get href
            jumpto href
            at new page:
            get body text. find keywords by regex. key words: 火灾、燃烧、火、爆炸、泄漏、渗漏 etc.
            get pdf links, download pdf

In [2]:

# basic info
# user_agent：
# Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36 Edg/105.0.1343.27
# chrome_path：
# C:\Program Files\Google\Chrome\Application
# base_url:
# https://www.msa.gov.cn/html/hxaq/sgjx/

from selenium import webdriver
from selenium.webdriver.common.by import By
import urllib
import logging
import os
import re
from retry import retry


# url = "https://www.msa.gov.cn/public/documents/document/mdqz/otex/~edisp/20220908043911094.pdf"
# file_dir = os.getcwd()
# file_name = url.split("/")[-1]
# file_path = os.path.join(file_dir, file_name)
# urllib.request.urlretrieve(url, file_path)


In [2]:
class RetryCounter():
    def __init__(self, tries = 3, exception_types = [Exception], logger = None) -> None:
        self.tries = tries
        self.exception_types = exception_types
        if logger is None:
            self.logger = logging.getLogger()
        else:
            self.logger = logger
        pass

    def exception_handler(self, exception):
        if self.tries == 0:
            self.logger.warning(f"{str(exception)}")
            return True
        flag_catch = False
        for elem in self.exception_types:
            if isinstance(exception, elem):
                flag_catch = True
                break
        if not flag_catch:
            self.logger.warning(f"{str(exception)}")
            return True
        self.tries -= 1
        return False

def log_error(exception):
    logger = logging.getLogger("errorlog")
    logger.warning()

def set_logger(log_file_path, logger):
    logFormatter = logging.Formatter("%(asctime)s %(message)s",
                                            "%m-%d %H:%M:%S")
    consoleHandler = logging.StreamHandler()
    consoleHandler.setFormatter(logFormatter)
    logger.addHandler(consoleHandler)
    
    fileHandler = logging.FileHandler(log_file_path)
    fileHandler.setFormatter(logFormatter)
    logger.addHandler(fileHandler)

In [3]:
# logging.basicConfig()
logger = logging.getLogger("myLogger")
set_logger("tmp.log", logger)
# tmpcnter = RetryCounter()

@retry(tries=3, logger=logger)
def func():
    raise ValueError("url info whatever")

func()


09-12 16:19:04 ValueError: url info whatever in __main__.func, retrying in 0 seconds...
09-12 16:19:04 ValueError: url info whatever in __main__.func, retrying in 0 seconds...


ValueError: url info whatever

In [4]:
@retry(tries=3, logger=logger)
def func():
    raise ValueError("url info whatever2")

func()

09-12 16:23:36 ValueError: url info whatever2 in __main__.func, retrying in 0 seconds...
09-12 16:23:36 ValueError: url info whatever2 in __main__.func, retrying in 0 seconds...


ValueError: url info whatever2

In [ ]:
<div class="text" id="ch_p">
		        	<p>2022年1月4日0355时，清远市某运输有限公司、张某强所有的清远籍自卸砂船“天XXXX8”船空载从广州新港开往珠海湾仔航行途中，与覃某新、邓某田所有的涉渔“三无”船在江门荷塘西江大桥下游约400米水域发生碰撞事故。碰撞事故造成涉渔“三无”船沉没，船上2人落水，其中1人获救，1人死亡。按照《水上交通事故统计办法》第六条、第二十九条的分级标准，该事故构成纳入统计的一般等级水上交通事故。<a href="https://www.msa.gov.cn/public/documents/document/mdqz/nje1/~edisp/20220525043615193.pdf" style="color: blue;" target="_blank">江门“1·4”“天XXXX8”船 与涉渔“三无”船碰撞事故调查报告（公开版）.pdf</a><br>
&nbsp;</p>
		        	<div class="foot_but">
			        	<a href="javascript:;" onclick="addFavorite()">收藏</a><a href="#" class="print_btn" onclick="doPrint()">打印本页</a><a href="javascript:;" onclick="closeWindow()">关闭窗口</a>
		        	</div>
		        </div>

In [1]:
import urllib.request
link = "https://www.msa.gov.cn/public/documents/document/mdqz/otex/~edisp/20220908043911094.pdf"
urllib.request.urlretrieve(link, "test.pdf")

('test.pdf', <http.client.HTTPMessage at 0x21eb3237eb0>)

In [3]:
dir_name = "dir"
file_name = "file.pdf"
os.path.join(dir_name, file_name)


'dir\\file.pdf'

In [9]:
chr(ord("D")+1)

'E'

In [5]:
# open base page
option = webdriver.ChromeOptions()
# option.add_argument('headless')
# driver = webdriver.Chrome(options=option)
driver = webdriver.Chrome()
base_url = "https://www.msa.gov.cn/html/hxaq/sgjx/"
driver.get(base_url)

In [10]:
all_handles = driver.window_handles
driver.switch_to.window(all_handles[1])

In [11]:
driver.close()

In [13]:
all_handles = driver.window_handles

In [14]:
all_handles

['CDwindow-2956062AD504FFDFB6A3D796BD4349CC']

In [16]:
driver.switch_to.window(all_handles[0])
print(driver.title)

航行安全 - 中华人民共和国海事局


In [1]:
print(driver.name)

NameError: name 'driver' is not defined

In [15]:
import re
text = "https://www.msa.gov.cn/public/documents/document/mdqz/otex/~edisp/20220908043911094.pdf"
regex = r"[^/]+(?!.*/)"
regex = re.compile(regex)
re.search(regex, text).group()

'20220908043911094.pdf'

In [17]:
from selenium import webdriver
# open base page
option = webdriver.ChromeOptions()
# option.add_argument('headless')
# driver = webdriver.Chrome(options=option)
driver = webdriver.Chrome()
base_url = "https://www.msa.gov.cn/html/hxaq/sgjx/"
driver.get(base_url)

In [21]:
from selenium.webdriver.common.by import By
last_slt = "#articleList > div > a.last"
total = driver.find_element(By.CSS_SELECTOR, last_slt).get_attribute("total")
cnt
active_slt = "#articleList > div > a.active"
active = driver.find_element(By.CSS_SELECTOR, active_slt).get_attribute("innerHTML")
active

'1'

: 

In [3]:
def get_keyword_regex(word_list):
    res = list()
    if not word_list:
        raise ValueError
    for word in word_list:
        if not word:
            continue
        res.append(f"{word}|")

    return "".join(res)[:-1]

# keword settings
word_list = [
    "起火",
    "火灾",
    "燃烧",
    "闪燃",
    "着火",
    "自燃",
    "爆燃",
    "爆炸",
    "溢油",
    "渗漏",
    "泄漏",
    "外漏",
    "外溢",
    "外泄",
    "污染",
]

res = get_keyword_regex(word_list)

In [8]:
import traceback
import logging
logging.basicConfig()
try:
    raise ValueError("agasd")
except Exception as e:
    logging.warning(traceback.format_exc())

  File "C:\Users\chen_\AppData\Local\Temp\ipykernel_78252\1842225033.py", line 5, in <module>
    raise ValueError("agasd")
ValueError: agasd



In [4]:
res

'起火|火灾|燃烧|闪燃|着火|自燃|爆燃|爆炸|溢油|渗漏|泄漏|外漏|外溢|外泄|污染'

In [13]:
import urllib.request
import requests
def get_status(url):
    res = requests.get(url)
    return res.status_code
url="https://www.msa.gov.cn/html/hxaq/sgjx/"
url = "https://www.baidu.com"
print(get_status(url))

200


In [10]:
import xlwings as xw
tmpxl_path = "data/result_all_old.xlsx"
tmpxl_path = "data/test.xlsx"
app = xw.App(visible=False, add_book=False)  # or something like xw.apps[10559] for existing apps, get the available PIDs via xw.apps.keys()

if os.path.exists(tmpxl_path):
    wb = app.books.open(tmpxl_path)
    sht = wb.sheets[0]
else:
    wb=app.books.add()
    #创建一个sheet工作表
    sht=wb.sheets['sheet1']
    # content
    # xl_all = list()
    xl_header = ["标题","id","url","本地链接","","","关键词上下文"]
    # xl_all.append(xl_header)
    # write to xl and save
    sht.range('A1').value = xl_header
    wb.save(tmpxl_path)

In [11]:
sht = wb.sheets[0]

In [13]:
cur_row = sht.used_range.last_cell.row + 1
cur_row

2

In [14]:
sht.range(f"A{cur_row}").value = "new"

In [15]:
wb.save(tmpxl_path)

In [16]:
app.quit()

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
# open base page
option = webdriver.ChromeOptions()
# option.add_argument('headless')
# driver = webdriver.Chrome(options=option)
driver = webdriver.Chrome()
base_url = "https://www.msa.gov.cn/html/hxaq/sgjx/"
driver.get(base_url)

# 创建action对象
action = webdriver.ActionChains(driver)
# action = ActionChains(driver)



In [10]:
driver.find_element(By.CSS_SELECTOR, "#pagInput").click()
# 对当前元素做输入
action.send_keys("21")
# 执行操作

action.perform()

In [8]:
bt_slt = 

In [11]:
driver.find_element(By.CSS_SELECTOR, "#articleList > div > input.btn.btn-fault").click()

In [12]:
driver.refresh()

In [15]:
textarea_slt = "body > div.c970 > div.con_wen > div:nth-child(3) > div.con_pl > textarea"
import re
class WordMatcher:
    def __init__(self, regex, context_expand = 8, exclude_chars = '\t\r\n') -> None:
        self.regex = regex
        self.context_expand = context_expand
        self.exclude_chars = exclude_chars
        pass

    def get_all_context(self, text):
        context_list = list()
        all_matches = re.finditer(self.regex, text)
        text_length = len(text)
        for match_elem in all_matches:
            context = text[max(match_elem.start()-self.context_expand, 0):min(match_elem.end()+self.context_expand, text_length)]
            if self.exclude_chars is not None:
                for ch in self.exclude_chars:
                    context = context.replace(ch, "")
            context_list.append(context)
        return context_list

def get_keyword_regex(word_list):
    res = list()
    if not word_list:
        raise ValueError
    for word in word_list:
        if not word:
            continue
        res.append(f"{word}|")

    return "".join(res)[:-1]

if __name__ == "__main__":
    word_list = [
    "起火",
    "火灾",
    "燃烧",
    "闪燃",
    "着火",
    "自燃",
    "爆燃",
    "爆炸",
    "溢油",
    "渗漏",
    "泄漏",
    "外漏",
    "外溢",
    "外泄",
    ]
    regex = get_keyword_regex(word_list)
    matcher = WordMatcher(regex)
    text = "sdfsasd"
    res = matcher.get_all_context(text)
    print()

False


In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
# open base page
option = webdriver.ChromeOptions()
# option.add_argument('headless')
# driver = webdriver.Chrome(options=option)
driver = webdriver.Chrome()
base_url = "https://www.baidu.com"
driver.get(base_url)
driver.get("http://www.safehoo.com/Case/Case/Blaze/")

driver.window_handles


['CDwindow-3C2B734F7A8710D891FEA8B463811E94']

In [8]:
driver.find_element(By.CSS_SELECTOR, "body > div.c970 > div.lm2_k650 > div.childclass_content > li:nth-child(1)").get_attribute("innerHTML")

'<span class="mor"><font color="red">08-31</font></span><img src="/images/common9.gif">\n                            [   文档]\n            <a href="/Case/Case/Blaze/202208/5682147.shtml" target="_blank" title="标题：湛江“8•6”“鑫华祥”轮火灾事故调查报告\r点击数：1\r发表时间：22年08月31日">湛江“8•6”“鑫华祥”轮火灾事故调查报告</a>'

In [9]:
driver.find_element(By.CSS_SELECTOR, "body > div.c970 > div.lm2_k650 > div.childclass_content > li:nth-child(1)").click()